# Download data
This notebook contains the code to download data from the [Nutritionix API](http://www.nutritionix.com/api) into a reasonable local data structure, create and structure a sqlite3 database, populate the database with data, and save the database to a file on disk.

The restaurant names and IDs have already been downloaded to `restaurant_ids.json`, and this task is beyond the scope of this notebook, so I'm just going to load the restaurant name/ID data from that file.

In [6]:
import requests
import json
import os
import minimum_sugar
import sqlite3
import types

# Load credential data from file
with open("credentials.json", "r") as f:
    credentials = json.load(f)
    
# Load restaurant IDs and names from file
with open("restaurant_ids.json", "r") as f:
    restaurants = json.load(f)

## Get data locally using Nutrionix API
The menu item data needs to be downloaded using the Nutritionix API and stored in a local data structure. This data structure will be a list of dicts, with each list item containing all the data Nutritionix has on that menu item.

In [7]:
# Could do some list comprehension here, but I think it code readability would suffer.
menu_data = []
for restaurant in restaurants:
    menu_data.extend(minimum_sugar.fetch_menu_item_data(restaurant["id"], credentials))

## Create SQLite database
The database needs to be created before it can be populated. The fields will correspond to the data fields provided by the Nutritionix API.

I found that [programmatically creating the table doesn't work like I expect](http://stackoverflow.com/questions/33019599/sqlite-why-cant-parameters-be-used-to-set-an-identifier). Thus, I have to explicitly write the `CREATE TABLE` string. I used the following python to help with the string creation, but I explicitly set the `CREATE TABLE` string in a subsequent cell.

```python
# This code gets me 95% of where I want to go regarding the `CREATE TABLE` command.
# I'll knock out the rest by hand.
schema_slug = "CREATE TABLE menu_data ({})"

# Map to [SQLite's types](https://www.sqlite.org/datatype3.html)
py2sql_map = {types.NoneType: "text",
    float: "real",
    int: "integer",
    unicode: "text",}

# Create dict with key and sql_key_type mapping
sql_keys_and_types = {key: py2sql_map[type(val)] for key, val in menu_data[0].iteritems()}

# Generate the string used to create the SQL table columns
sql_table_columns_command = ",\n".join([" ".join(item) for item in sql_keys_and_types.iteritems()])

# Pretty-print stuff.
max_col_name_len = max([len(key) for key in sql_keys_and_types.keys()])
sql_table_columns_command_lines = ['{:<{}s} {}'.format(key, max_col_name_len, val) for key, val in sql_keys_and_types.iteritems()]

create_table_cmd = schema_slug.format(",\n".join(sql_table_columns_command_lines))

print create_table_cmd
```

In [31]:
create_table_cmd = """CREATE TABLE menu_data (
    item_id                     text,
    nf_ingredient_statement     text,
    nf_serving_weight_grams     integer,
    brand_name                  text,
    nf_calories_from_fat        integer,
    nf_trans_fatty_acid         integer,
    brand_id                    text,
    nf_sodium                   integer,
    allergen_contains_eggs      text,
    nf_iron_dv                  integer,
    nf_cholesterol              integer,
    item_description            text,
    nf_monounsaturated_fat      text,
    nf_dietary_fiber            integer,
    item_type                   integer,
    item_name                   text,
    allergen_contains_tree_nuts text,
    allergen_contains_shellfish text,
    nf_vitamin_c_dv             integer,
    nf_polyunsaturated_fat      text,
    allergen_contains_peanuts   text,
    nf_sugars                   integer,
    nf_servings_per_container   text,
    nf_total_fat                integer,
    nf_total_carbohydrate       integer,
    nf_saturated_fat            integer,
    allergen_contains_wheat     text,
    updated_at                  text,
    allergen_contains_gluten    text,
    nf_protein                  integer,
    nf_calories                 integer,
    nf_water_grams              text,
    allergen_contains_fish      text,
    nf_calcium_dv               integer,
    nf_serving_size_qty         real,
    allergen_contains_milk      text,
    nf_vitamin_a_dv             integer,
    nf_serving_size_unit        text,
    nf_refuse_pct               text,
    allergen_contains_soybeans  text
)"""